In [40]:
"""4.15 本次使劲紧迫，暂时只对JD的【详细信息】和CV中的不同【工作经验】进行匹配"""
import os
import pandas as pd
import json
import match
from match import AvgWvSim,FocusedAvgWvSim


print("===========当前分析的jd为：ApplicationsSpecialist===========")
JD_path = r'F:\Jupyter\--NLP\JD_CV_match\superHR\ner_interface\output\jd\\
201804-101824_ApplicationsSpecialist_337332_entity.json'
with open(JD_path,encoding='utf-8') as f:
    JD_words = json.loads(f.read())
    print("@@ JD抽取的实体信息：")
    for each in JD_words:
        print(each,' : ',JD_words[each])
jd_entity = JD_words['详细信息'] #################本次只取这一条####################

# ===========================获取真实排序，然后据此挑出对应的cv：=======================
truth = pd.read_excel('data/roche100_id_score.xlsx',sheet='ApplicationsSpecialist')
true_rank = truth.sort_values(by='score',ascending=False)
true_rank_list = list(true_rank.id)


print("-----读取对应的100份CV......")
CVs_path = r'F:\Jupyter\--NLP\JD_CV_match\superHR\ner_interface\output\cv'
# fs = os.listdir(CVs_path)
# jsons = [f for f in fs if f[-12:]=='_entity.json']
CVs_words = []
for cv_id in true_rank_list:
    with open(CVs_path+'/'+cv_id+'_entity.json',encoding='utf-8') as f:
        l = json.loads(f.read())
        CVs_words.append(l)
print("-----读取完毕！ 开始进行分析！------")
print(CVs_words[1],'-------------')
print('JD items:\n',JD_words.keys(),'\n')
print('JD items中的【详细信息】:\n',JD_words['详细信息'],'\n')
print('CV items:\n',CVs_words[1].keys(),'\n')
print('CV items中的【工作经验】项:\n',CVs_words[1]['工作经验'][0])
print(len(CVs_words))

===========当前分析的jd为：ApplicationsSpecialist===========
@@ JD抽取的实体信息：
id  :  337332
name  :  201804-101824  Applications Specialist
职能  :  ['售前', '售后技术支持工程师']
学历  :  本科
年限  :  2年
省  :  福建
市  :  福州
专业  :  
技能  :  
业务  :  
行业  :  
产品  :  
详细信息  :  {'技能': ['客户(经销商', '英语口语', '书面表达', '微软办公软件', '外企销售'], '学历': ['大学'], '专业': ['医学检验专业'], '行业': [], '职能': [], '年限': ['两年'], '业务': ['终端客户)', '产品培训', '新产品上市', '内部销售', '工程师', '经销商提供产品', '培训结果反馈', '配合维修', '用户进行培训', '产品信息', '客户解释测试结果'], '产品': []}
-----读取对应的100份CV......
-----读取完毕！ 开始进行分析！------
{'in_path': 'D:/Project/NER_Interface_v3_20190412/ner_interface/data/CV_Origin_Data/201804-101824ApplicationsSpecialist/58b6b4ffb1222c305523f069.json', 'id': '58b6b4ffb1222c305523f069', '工作年限': 8.0, '工作城市': ["['福州']"], '专业': ['分子生物学', '生物技术'], '毕业院校': ['华南理工大学', '韶关大学'], '学历': ['硕士', '本科'], '工作经验': [{'工作公司': '西门子医疗系统有限公司', '工作行业': ['医疗设备', '器械'], '工作部门': ['影像设备'], '工作职位': ['客户经理'], '岗位工作年限': 0.5, '工作描述': {'技能': [], '学历': [], '专业': [], '行业': [], '职能': [], '年限': [], '业

In [41]:
"""
现在面对的标签更加繁多了。。。得理一理：
之前的7个标签可以对上的是JD中的【详细信息】，和CV中【工作经验】中的【工作描述】；
另外，7个标签中：
- ‘学历’和‘年限’不宜通过词向量比较，因此要单独拿出来比较大小。
但是不满足的话打几分？满足的话打几分？这个标准需要订立下来。
也可以这样，满足就是1，否则0，或者算一个gap；然后再乘上学历、年限的权重。
- 专业如果能构建一个知识图谱，或者，简单的一个层次图，就可以和明确地衡量相似度：
先用词向量的方法寻找图谱上最相近专业，然后图谱上各个专业有明确的规则来表示距离。通过词向量直接衡量，感觉精确度不高；
- 技能，无法用词向量的方式来衡量相似度，尤其是编程领域类的技能
（能够训练一个细分领域的词向量，可以拉开不同编程语言的距离？）或者，能不能先不衡量相似度：
如果有编程语言要求，先筛选一批明显不满足的（招java，结果cv全篇没提java）；其他的，不计算相似度，
知识图谱也不要指望构建，没有标签，不同语言交叉太多；而是把这些技能词汇收集起来，形成一个供HR选择集合，
HR轻轻一点，就可以筛选掉很多词，？？？
- 职位/职能（么有区别吧），可以用词向量试试吧
- 行业，可以用词向量试试。但这里我还有一个疑问，行业能否穷举？行业如果没有写，其实也可以通过主题识别的方式来获取；
- 业务，是我最搞不懂的一个标签。提取出来的很多东西，根本不知道是什么东西，不如直接拿原来的句子进行匹配；

"""

tag_weights = {'专业':0.2,'业务':0.2,'学历':0.1,'年限':0.2,'技能':0.2,'职能':0,'行业':0.1,'产品':0}
match_result = {}
match_result['cv_id'] = []
match_result['score'] = []
for cv in CVs_words:
    """要区分不同的工作经历"""
    cv_id = cv['id']
    print('--------')
    print('【cv_id】:',cv_id)
    try:
        exps = cv['工作经验']
        """
        如何衡量不同工作经历的权重（与jd的相似度、时间的远近）？
        直接每一条去算一个相似度，相似度越大的，自然权重也越大，可以让自身作为一个权重，
        所以可以直接求各个相似度的加时间权的平方和。时间权重应该设置一个衰减系数α(0~1).
        ——但这样就有个问题，工作经验条数越大，最后的加权和一般也会更大。
        所以==>只拿相似度最大的作为分值，其他的忽略。但时间权重可以加，毕竟越早的经历越不重要。
        """
        time_decay = 0.95 #实际更复杂，还要考虑工作年限
        scores = []
        for i,exp in enumerate(exps):
            score = FocusedAvgWvSim(jd_entity, exp['工作描述'],tag_weights=tag_weights)
            score = pow(time_decay,i+1)*score
            scores.append(score)
        print('@ Score:',max(scores))

        match_result['cv_id'].append(cv_id)
        match_result['score'].append(max(scores))
    except:
        print('该cv中没有【工作经验】，请复核！')
        match_result['cv_id'].append(cv_id)
        match_result['score'].append(0)
df = pd.DataFrame(match_result)
show_df = pd.concat([df['cv_id'],df['score']],axis=1)
print('\n==============各CV排序：==============')
predict_rank = show_df.sort_values(by='score',ascending=False)
predict_rank.to_csv('data/predict_result.csv')
pd.concat([true_rank.reset_index(),predict_rank.reset_index()],axis=1).to_csv('data/duizhao.csv')
true_rank

--------
【cv_id】: 5a1c0290f0f0f24d856eb2aa
@ Score: 0.4023014456581658
--------
【cv_id】: 58b6b4ffb1222c305523f069
@ Score: 0.3190929238082673
--------
【cv_id】: 5b0bae97611c023fbec67c9e
@ Score: 0.5123500916579906
--------
【cv_id】: 5b06c5ec611cc7db31677456
@ Score: 0.34846789693939156
--------
【cv_id】: 595c7c79f0f0dcff9b039f26
@ Score: 0.3038142576693557
--------
【cv_id】: 5b02d273611cd9a3b1eac331
@ Score: 0.30258728206981145
--------
【cv_id】: 58e5ae75f0f014cf1e370797
@ Score: 0.4112622769349606
--------
【cv_id】: 5b3c2680611c19891c98a385
@ Score: 0.4019834399096605
--------
【cv_id】: 5a4ae751f0f079b703308ab5
@ Score: 0.4814860878427447
--------
【cv_id】: 5aa7f426f0f074046641c495
@ Score: 0.33836625312827046
--------
【cv_id】: 5b2bde2d611cf2b142c1d06f
@ Score: 0.2150352394057617
--------
【cv_id】: 5961b469f0f0d91c94872d5a
@ Score: 0.3339383894396953
--------
【cv_id】: 5bd17c79f0f0111376f855bb
@ Score: 0.2924664363325612
--------
【cv_id】: 5ba90ed8f0f00da092b62673
@ Score: 0.2873599338088615
---

,id,score
0,5a1c0290f0f0f24d856eb2aa,91
7,58b6b4ffb1222c305523f069,87
16,5b0bae97611c023fbec67c9e,86
27,5b06c5ec611cc7db31677456,85
89,595c7c79f0f0dcff9b039f26,85
20,5b02d273611cd9a3b1eac331,84
77,58e5ae75f0f014cf1e370797,83
22,5b3c2680611c19891c98a385,83
1,5a4ae751f0f079b703308ab5,83
6,5aa7f426f0f074046641c495,82


In [36]:
import numpy as np
## test bed:
def topn_result(l1,l2,n):
    set1 = set(l1[:n])
    set2 = set(l2[:n])
    inter = set1.intersection(set2)
    return [len(inter)/n,list(inter)]

predict_rank_list = list(predict_rank.cv_id) 
for n in [5,10,20,30,100]:
    [score,correct_items] = topn_result(true_rank_list,predict_rank_list,n)
    print('\n===============\nTop-%d score:'%n,score)
    print('---------------\n详细结果：')
    for item in true_rank_list[:n]:
        if item in correct_items:
            label = '√'
        else:
            label = ''
        print('CV id:',item,' ',label)


Top-5 score: 0.0
---------------
详细结果：
CV id: 5a1c0290f0f0f24d856eb2aa   
CV id: 58b6b4ffb1222c305523f069   
CV id: 5b0bae97611c023fbec67c9e   
CV id: 5b06c5ec611cc7db31677456   
CV id: 595c7c79f0f0dcff9b039f26   

Top-10 score: 0.2
---------------
详细结果：
CV id: 5a1c0290f0f0f24d856eb2aa   
CV id: 58b6b4ffb1222c305523f069   
CV id: 5b0bae97611c023fbec67c9e   √
CV id: 5b06c5ec611cc7db31677456   
CV id: 595c7c79f0f0dcff9b039f26   
CV id: 5b02d273611cd9a3b1eac331   
CV id: 58e5ae75f0f014cf1e370797   
CV id: 5b3c2680611c19891c98a385   
CV id: 5a4ae751f0f079b703308ab5   √
CV id: 5aa7f426f0f074046641c495   

Top-20 score: 0.35
---------------
详细结果：
CV id: 5a1c0290f0f0f24d856eb2aa   
CV id: 58b6b4ffb1222c305523f069   
CV id: 5b0bae97611c023fbec67c9e   √
CV id: 5b06c5ec611cc7db31677456   
CV id: 595c7c79f0f0dcff9b039f26   
CV id: 5b02d273611cd9a3b1eac331   
CV id: 58e5ae75f0f014cf1e370797   √
CV id: 5b3c2680611c19891c98a385   √
CV id: 5a4ae751f0f079b703308ab5   √
CV id: 5aa7f426f0f074046641c495

In [14]:
l = [1,2,3]
max(l)

3